In [4]:
# 지마켓 Best Seller 상품 정보 수집하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy
import urllib.request
import urllib
import pandas as pd   
import xlwt 
import random
import os


print("=" *80)
print("지마켓 Best Seller 상품 정보 추출하기")
print("=" *80)

query_txt='지마켓'
query_url='https://corners.gmarket.co.kr/Bestsellers'

cnt = int(input('  1.크롤링 할 건수는 몇건입니까?: '))

f_dir = input("  2.파일이 저장될 경로만 쓰세요; (예:c:\\temp\\):")
print("\n")

if cnt > 30 :
      print("    요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")
      
# Step 3. 저장될 파일위치와 이름을 지정 한 후 크롬 드라이버를 실행하여 페이지를 엽니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_dir=f_dir+s+'-'+query_txt
ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

path = "c:/practice/chromedriver.exe"
driver = webdriver.Chrome(path)
    
driver.get(query_url)
time.sleep(5)

#Step 4. 화면을 스크롤해서 아래로 이동한 후 요청된 데이터를 수집합니다.

def scroll_down(driver):
      
      driver.execute_script("window.scrollBy(0,9300);")
      time.sleep(1)

scroll_down(driver)

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만듭니다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 이미지 추출 코드 추가
img_src2=[]   # 이미지 URL
file_no = 0


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

reple_result = soup.select('div.best-list')
slist = reple_result[1].find_all('li')

if cnt < 100 :         
    ranking2=[]
    title3=[]
    price2=[]
    price3=[]
    sale2=[]
    store2=[]
       
    count = 0
    
    # 이미지 저장용 폴더 생성하기
    img_dir = ff_dir+"\\images"
    os.makedirs(img_dir)
    os.chdir(img_dir)

    for li in slist:
            
            # 이미지 저장하기
            try :
              photo = li.find('div', 'thumb').find('img')['src']
            except AttributeError :
              continue
            file_no += 1

            urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
            time.sleep(1)

            if cnt == file_no :
              break
            
            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")

            # 판매순위
            print("-" *70)
            try :
             ranking = li.find('p').get_text().replace("#","")
            except AttributeError :
             ranking = ''
             print(ranking.replace("#",""))
            else :
             print("1.판매순위:",ranking)

             f.write('1.판매순위:'+ ranking + "\n")

            #제품 소개 
            try :
             title1 = li.find('a', 'itemname').get_text().replace("\n","")
            except AttributeError :
             title1 = ''
             print(title1.replace("\n",""))
             f.write('2.제품소개:'+ title1 + "\n")
            else :
             title2=title1.translate(bmp_map).replace("\n","") 
             print("2.제품소개:", title2.replace("\n",""))

             count += 1
             
             f.write('2.제품소개:'+ title2 + "\n")
            
             # 원래가격
             try :
               price = li.find('div', 'o-price').find('span').get_text().replace("\n","")
             except AttributeError :
               price = ''
               
             print("3.원래가격:", price.replace("\n",""))
             f.write('3.원래가격:'+ price + "\n")
                
             # 판매가격     
             try :
                dc_price = li.find('div', 's-price').find('span').get_text().replace(",","")
             except (IndexError , AttributeError) :
                dc_price=''
                print('4.판매가격: ',dc_price)
                f.write('4.판매가격:'+ dc_price + "\n")
             else :
                print('4.판매가격:',dc_price)
                f.write('4.판매가격:'+ dc_price + "\n")

            #할인율
             try :
                sale = li.find('div', 's-price').find('em').get_text().replace(",","")
             except (IndexError , AttributeError) :
                sale =''
                print('5.할인율: ',sale)
                f.write('5.할인율:'+ sale + "\n")
             else :
                print('5.할인율:',sale)
                f.write('5.할인율:'+ sale + "\n")

             print("-" *70)
                          
             f.close( )
              
             time.sleep(0.3)
            
             ranking2.append(ranking)
             title3.append(title2.replace("\n",""))
             price2.append(price.replace("\n",""))
             price3.append(dc_price.replace("\n",""))
             sale2.append(sale.replace("\n",""))

             if count == cnt :
                break
            
else :
      print(" 검색 건수는 1건 - 최대 100 건까지만 가능합니다")               
                          

  

#Step 5. 검색 결과를 다양한 형태로 저장하기
              
gmarket_best_seller = pd.DataFrame()
gmarket_best_seller['판매순위']=ranking2
gmarket_best_seller['제품소개']=pd.Series(title3)
gmarket_best_seller['원래가격']=pd.Series(price2)
gmarket_best_seller['판매가격']=pd.Series(price3)
gmarket_best_seller['할인율']=pd.Series(sale2)


# csv 형태로 저장하기
gmarket_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
gmarket_best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f


#이미지 삽입하기

import win32com.client as win32
import win32api  
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()

# Step 6. 요약 정보를 출력하기
print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 상품 중에서 실제 크롤링 된 상품수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)    

지마켓 Best Seller 상품 정보 추출하기
  1.크롤링 할 건수는 몇건입니까?: 5
  2.파일이 저장될 경로만 쓰세요; (예:c:\temp\):c:\practice\


    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~
----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: [바다원]건오징어 5미
3.원래가격: 26,800원
4.판매가격: 13400원
5.할인율: 50%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: AHC 유스래스팅 리얼 아이크림 포 페이스 시즌9 30ml 4개 +12ml
3.원래가격: 
4.판매가격: 27500원
5.할인율:  
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: [동원에프앤비]동원 인포켓 치즈 오리지널 (스트링치즈) x40개 + 증정
3.원래가격: 32,100원
4.판매가격: 22500원
5.할인율: 29%
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 4
2.제품소개: [애경트리오]주방세제 트리오 베이킹소다 담은 1.2Lx5개+수세미))
3.원래가격: 20,000원
4.판매가격: 10000원
5.할인율: 50%
